In [1]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import random

import pandas as pd
import numpy as np
import os

## Create test data

In [4]:
import json
data = json.load(open("./addresses.json"))
df_addresses = pd.DataFrame(data["addresses"])
df_states = pd.read_csv("./states.csv", sep=";")
df_states.columns=["state_name", "state"]
df = pd.merge(df_addresses, df_states, on="state")[["state_name", "city", "postalCode"]].astype(str)
df.columns=["state", "city", "location"]
df["value"] = np.random.randint(100,1000,len(df))
df.insert(loc=0, column="id", value=range(0, len(df)))
df.to_csv("./locations.csv", encoding="utf-8", sep="|")

In [3]:
df.tail(2)

,id,state,city,location,value
3025,3025,Florida,Panama City Beach,32408,957
3026,3026,Florida,Panama City Beach,32407,172


## Create database

In [124]:
con = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_DATAPRODUCT_USER"),
    password=os.getenv("SNOWFLAKE_DATAPRODUCT_PASSWORD"),
    account=(
        os.getenv("SNOWFLAKE_DATAPRODUCT_ACCOUNT")
        + "."
        + os.getenv("SNOWFLAKE_DATAPRODUCT_REGION")
    ),
    role="vdl_dataproduct_role",
    warehouse="vdl_dataproduct_warehouse"
)

In [127]:
def create_warehouse_database_and_schema(db):
    db.execute("CREATE WAREHOUSE IF NOT EXISTS test_warehouse")
    db.execute("CREATE DATABASE IF NOT EXISTS testdb")
    db.execute("USE DATABASE testdb")
    db.execute("CREATE SCHEMA IF NOT EXISTS testschema")
    db.execute("CREATE SCHEMA IF NOT EXISTS security")

    db.execute("USE WAREHOUSE test_warehouse")
    db.execute("USE SCHEMA testdb.testschema")
    db.execute(f"USE ROLE vdl_dataproduct_role")


def drop_warehouse_database_and_schema(db):
    db.execute("DROP SCHEMA IF EXISTS testschema")
    db.execute("DROP DATABASE IF EXISTS testdb")
    db.execute("DROP WAREHOUSE IF EXISTS test_warehouse")



In [128]:
with con.cursor() as cur:
    create_warehouse_database_and_schema(cur)

In [165]:
df.head(1)

,id,state,city,location,value
0,0,District of Columbia,Washington,20020,2892


In [308]:
with con.cursor() as cur:

    sql = """
            create or replace table transactions (
            id       number,
            state    varchar,
            city     varchar,
            location varchar,
            value    number       
            );
        """

    cur.execute(sql)

    sql = """
            create or replace table security.states (
            role     varchar,
            state    varchar
            );
        """

    cur.execute(sql)

    sql = """
            create or replace table security.cities (
            role    varchar,
            city    varchar
            );
        """

    cur.execute(sql)

    sql = """
            create or replace table security.locations (
            role      varchar,
            location  varchar
            );
        """
        
    cur.execute(sql)

In [309]:

success, nchunks, nrows, _ = write_pandas(con, df, "TRANSACTIONS", quote_identifiers=False)
print(success, nchunks, nrows)

True 1 3027


In [310]:
states = df["state"].unique()
df_states = pd.DataFrame()
df_states["role"] = states
df_states["state"] = states
success, nchunks, nrows, _ = write_pandas(con, df_states, "STATES", schema="SECURITY", quote_identifiers=False)
print(success, nchunks, nrows)

True 1 15


In [311]:
cities = df["city"].unique()
df_cities = pd.DataFrame()
df_cities["role"] = cities
df_cities["city"] = cities
success, nchunks, nrows, _ = write_pandas(con, df_cities, "CITIES", schema="SECURITY", quote_identifiers=False)
print(success, nchunks, nrows)

True 1 414


In [312]:
locations = df["location"].unique()
df_location = pd.DataFrame()
df_location["role"] = locations
df_location["location"] = locations
success, nchunks, nrows, _ = write_pandas(con, df_location, "LOCATIONS", schema="SECURITY", quote_identifiers=False)
print(success, nchunks, nrows)

True 1 667


In [313]:
sql = """
    create or replace row access policy security.transactions_policy as (state_name varchar, city_name varchar, location_name varchar) returns boolean ->
    exists (
            select 1 from security.states
            where upper(role) in (select upper(NAME) from security.current_roles where ROLE_NAME = current_role())
                and state = state_name
    )
    or exists (
        select 1 from security.cities
        where upper(role) in (select upper(NAME) from security.current_roles where ROLE_NAME = current_role())
            and city = city_name
    )
    or exists (
        select 1 from security.locations
        where upper(role) in (select upper(NAME) from security.current_roles where ROLE_NAME = current_role())
            and location = location_name
    )
    ;
"""

with con.cursor() as cur:
    cur.execute(sql)

In [314]:
with con.cursor() as cur:
    cur.execute("alter table transactions add row access policy security.transactions_policy on (state, city, location);")

""" HACK

Snapshot all roles granted to the user or role:

show grants to role vdl_dataproduct_role;
create or replace table testdb.security.current_roles as with roles as (select "name"
from table(result_scan(last_query_id()))
where "granted_on" = 'ROLE') select 'VDL_DATAPRODUCT_ROLE' as role_name,"name" as name from roles;

## Test policy

In [319]:
with con.cursor() as cur:
    sql = "select * from security.current_roles"
    cur.execute(sql)
    df_res = cur.fetch_pandas_all()

df_res

,ROLE_NAME,NAME
0,VDL_DATAPRODUCT_ROLE,ALASKA
1,VDL_DATAPRODUCT_ROLE,WASHINGTON


In [321]:
with con.cursor() as cur:
    sql = "select * from transactions"
    cur.execute(sql)
    df_res = cur.fetch_pandas_all()

print(df_res.head(10))
print(df_res.tail(10))



   ID                 STATE        CITY LOCATION  VALUE
0   0  District of Columbia  Washington    20020   2892
1   1  District of Columbia  Washington    20001   2892
2   2  District of Columbia  Washington    20011   2892
3   3  District of Columbia  Washington    20016   2892
4   4  District of Columbia  Washington    20003   2892
5   5  District of Columbia  Washington    20012   2892
6   6  District of Columbia  Washington    20003   2892
7   7  District of Columbia  Washington    20002   2892
8   8  District of Columbia  Washington    20001   2892
9   9  District of Columbia  Washington    20011   2892
       ID   STATE       CITY LOCATION  VALUE
361  2447  Alaska  Anchorage    99507   2892
362  2448  Alaska  Anchorage    99577   2892
363  2449  Alaska  Anchorage    99577   2892
364  2450  Alaska  Anchorage    99518   2892
365  2451  Alaska  Anchorage    99507   2892
366  2452  Alaska  Anchorage    99504   2892
367  2453  Alaska  Anchorage    99577   2892
368  2454  Alaska  Ancho

## Cleanup

In [334]:
with con.cursor() as cur:
    drop_warehouse_database_and_schema(cur)